In [1]:
import pandas as pd
import os
import sys
from io import StringIO
from datetime import datetime, timedelta, time
#from scipy.integrate import trapezoid
import pvlib
import matplotlib.pyplot as plt  # for visualization
import statistics

#only needed for jupyter file path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from components.SmartPowerStation import SmartPowerStation, Controls

In [2]:
CONTROLS = Controls()
CONTROLS.url = '192.168.1.116'

In [49]:
batWh = 204.8
batACW = 300
pvW = 50

In [4]:
recentDays = await CONTROLS.getRecentData(10)

In [5]:
#merge files
def mergeDays(files):
    merged = files[-1].copy()
    for index, file in reversed(list(enumerate(files))):
        #print(index)
        if index != len(files):
            merged = pd.concat([merged,file], ignore_index=True)
    return merged

allData = mergeDays(recentDays)
allData.tail()

,datetime,powerstation_percentage,powerstation_inputWAC,powerstation_inputWDC,powerstation_outputWAC,powerstation_outputWDC,powerstation_outputMode,powerstation_deviceType,relay1_power,relay1_current,...,relay2_voltage,relay2_status,relay2_device,relay3_power,relay3_current,relay3_voltage,relay3_status,relay3_device,mode,position
6032,2025-05-12 09:22:28.429441,70.0,0.0,0.0,0.0,0.0,40.0,AC2A,1.6,0.039,...,121.6,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B
6033,2025-05-12 09:24:56.444558,70.0,0.0,0.0,0.0,0.0,40.0,AC2A,1.5,0.038,...,122.0,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B
6034,2025-05-12 09:27:23.456139,70.0,0.0,0.0,0.0,0.0,40.0,AC2A,1.6,0.038,...,121.8,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B
6035,2025-05-12 09:29:49.423790,69.0,0.0,0.0,0.0,0.0,40.0,AC2A,1.6,0.039,...,121.4,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B
6036,2025-05-12 09:32:16.437130,69.0,0.0,1.0,0.0,0.0,41.0,AC2A,80.0,0.700,...,121.6,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B


## Measure Self-Consumption

Find chunks of time where DC in, AC output, and AC input are zero; measure % change over time; and average it

In [6]:
sc_DF = allData[(allData['relay3_power'] == 0) & (allData['relay2_power'] == 0) & (allData['powerstation_inputWDC'] == 0) & (allData['powerstation_outputWDC'] == 0) ][['datetime','powerstation_percentage','powerstation_inputWDC','relay2_power','relay3_power']]

In [7]:
sc_DF.head()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_power,relay3_power
1,2025-05-03 00:06:15.391036,77.0,0.0,0.0,0.0
2,2025-05-03 00:08:42.417400,77.0,0.0,0.0,0.0
3,2025-05-03 00:11:07.387912,77.0,0.0,0.0,0.0
4,2025-05-03 00:13:33.417699,77.0,0.0,0.0,0.0
5,2025-05-03 00:16:00.382352,77.0,0.0,0.0,0.0


In [8]:
sc_DF.tail()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_power,relay3_power
6031,2025-05-12 09:20:01.444401,70.0,0.0,0.0,0.0
6032,2025-05-12 09:22:28.429441,70.0,0.0,0.0,0.0
6033,2025-05-12 09:24:56.444558,70.0,0.0,0.0,0.0
6034,2025-05-12 09:27:23.456139,70.0,0.0,0.0,0.0
6035,2025-05-12 09:29:49.423790,69.0,0.0,0.0,0.0


In [9]:
sc_DF.index

Index([   1,    2,    3,    4,    5,    6,    9,   10,   11,   12,
       ...
       6026, 6027, 6028, 6029, 6030, 6031, 6032, 6033, 6034, 6035],
      dtype='int64', length=3931)

In [10]:
sc_DF.index[23:24]

Index([27], dtype='int64')

In [11]:
#to do: check if dates are contiguous!
def contiguousTimeChunks(df):
    chunks = []
    startIndex = df.index[0]
    prevIndex = startIndex -1
    temp = df.index
    for i in temp:
        #print(i)
        if prevIndex + 1 != i:   # change -1 to 25      
            chunks.append(df.loc[startIndex:prevIndex])
            startIndex = i
        if i == temp[-1]:
            print(i)
            chunks.append(df.loc[startIndex:i])
        prevIndex = i

    return chunks

sc_List = contiguousTimeChunks(sc_DF)

sc_ListF = []        
# filter out lists with only 0 values
for l in sc_List:
    pc = l['powerstation_percentage'].unique() # should have at least 2 drops in percentage
    if len(pc) > 2:
        sc_ListF.append(l)

6035


In [12]:
#convert percentage change to Wh
def WhChange(pc, bwh=batWh):
    return (bwh * (pc * .01))

In [13]:
scAn = pd.DataFrame(columns=['startDT', 'endDT','len','duration','durationS','durationH','percChange','wHChange'])

for c in sc_ListF:
    perc_change = c.iloc[0]['powerstation_percentage']- c.iloc[-1]['powerstation_percentage']
    wH_change = WhChange(perc_change, batWh) 
    dH = c.iloc[-1]['datetime'] - c.iloc[0]['datetime']
    df = pd.DataFrame({'startDT': [c.iloc[0]['datetime']], 'endDT': [c.iloc[-1]['datetime']],'len':[len(c)],'duration':[dH],'durationS':[dH.total_seconds()],'durationH':[dH.total_seconds()/60/60],'percChange':[perc_change],'wHChange':[wH_change]})
    scAn = pd.concat([scAn,df], ignore_index=True)

# scAn = pd.DataFrame(scAn).set_index('datetime')

# # Resample by 1 hour and compute the average
# hourly_avg = df.resample('1H').mean()

# print(hourly_avg)

C:\Users\alexn\AppData\Local\Temp\ipykernel_5884\1584439748.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  scAn = pd.concat([scAn,df], ignore_index=True)


In [14]:
#filter out where percChange is less than 1
scAn = scAn[scAn['percChange'] >= 1]
scAn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   startDT     137 non-null    datetime64[ns] 
 1   endDT       137 non-null    datetime64[ns] 
 2   len         137 non-null    object         
 3   duration    137 non-null    timedelta64[ns]
 4   durationS   137 non-null    float64        
 5   durationH   137 non-null    float64        
 6   percChange  137 non-null    float64        
 7   wHChange    137 non-null    float64        
dtypes: datetime64[ns](2), float64(4), object(1), timedelta64[ns](1)
memory usage: 8.7+ KB


In [15]:
scAn.head()

,startDT,endDT,len,duration,durationS,durationH,percChange,wHChange
0,2025-05-03 00:56:04.419714,2025-05-03 01:57:49.421440,26,0 days 01:01:45.001726,3705.001726,1.029167,4.0,8.192
1,2025-05-03 02:03:26.392317,2025-05-03 02:40:12.400608,16,0 days 00:36:46.008291,2206.008291,0.612780,3.0,6.144
2,2025-05-03 02:47:41.398473,2025-05-03 03:51:25.761991,27,0 days 01:03:44.363518,3824.363518,1.062323,4.0,8.192
3,2025-05-03 03:58:57.394002,2025-05-03 05:02:42.389669,27,0 days 01:03:44.995667,3824.995667,1.062499,4.0,8.192
4,2025-05-03 05:45:32.433874,2025-05-03 06:27:13.425963,18,0 days 00:41:40.992089,2500.992089,0.694720,3.0,6.144


In [16]:
scAn['scW']= scAn['wHChange']/scAn['durationH']
scAn['scP']= scAn['percChange']/scAn['durationH']


In [17]:
scAn

,startDT,endDT,len,duration,durationS,durationH,percChange,wHChange,scW,scP
0,2025-05-03 00:56:04.419714,2025-05-03 01:57:49.421440,26,0 days 01:01:45.001726,3705.001726,1.029167,4.0,8.192,7.959834,3.886638
1,2025-05-03 02:03:26.392317,2025-05-03 02:40:12.400608,16,0 days 00:36:46.008291,2206.008291,0.612780,3.0,6.144,10.026436,4.895720
2,2025-05-03 02:47:41.398473,2025-05-03 03:51:25.761991,27,0 days 01:03:44.363518,3824.363518,1.062323,4.0,8.192,7.711401,3.765332
3,2025-05-03 03:58:57.394002,2025-05-03 05:02:42.389669,27,0 days 01:03:44.995667,3824.995667,1.062499,4.0,8.192,7.710126,3.764710
4,2025-05-03 05:45:32.433874,2025-05-03 06:27:13.425963,18,0 days 00:41:40.992089,2500.992089,0.694720,3.0,6.144,8.843850,4.318286
...,...,...,...,...,...,...,...,...,...,...
132,2025-05-11 21:10:25.214142,2025-05-11 21:44:54.248303,15,0 days 00:34:29.034161,2069.034161,0.574732,2.0,4.096,7.126804,3.479885
133,2025-05-11 23:42:45.212014,2025-05-12 03:57:50.455090,103,0 days 04:15:05.243076,15305.243076,4.251456,26.0,53.248,12.524649,6.115551
134,2025-05-12 04:03:26.450455,2025-05-12 06:51:05.409449,69,0 days 02:47:38.958994,10058.958994,2.794155,10.0,20.480,7.329586,3.578899
135,2025-05-12 07:01:32.410235,2025-05-12 09:00:03.438113,49,0 days 01:58:31.027878,7111.027878,1.975286,7.0,14.336,7.257685,3.543791


In [18]:
scAn.describe()

,startDT,endDT,duration,durationS,durationH,percChange,wHChange,scW,scP
count,137,137,137,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000
mean,2025-05-07 03:18:27.352845312,2025-05-07 04:19:02.181602816,0 days 01:00:34.828757372,3634.828757,1.009675,4.372263,8.954394,9.002764,4.395881
min,2025-05-03 00:56:04.419714,2025-05-03 01:57:49.421440,0 days 00:14:43.983856,883.983856,0.245551,2.000000,4.096000,5.886418,2.874227
25%,2025-05-04 17:13:26.145846016,2025-05-04 18:25:27.114771968,0 days 00:39:13.000541,2353.000541,0.653611,3.000000,6.144000,7.992256,3.902469
50%,2025-05-06 22:16:25.953239040,2025-05-06 23:52:41.939427072,0 days 00:56:26.988931,3386.988931,0.940830,4.000000,8.192000,8.638282,4.217911
75%,2025-05-09 20:44:07.075160064,2025-05-09 21:11:05.102842112,0 days 01:03:44.363518,3824.363518,1.062323,4.000000,8.192000,9.346537,4.563739
max,2025-05-12 09:07:44.413008,2025-05-12 09:29:49.423790,0 days 04:15:05.243076,15305.243076,4.251456,26.000000,53.248000,16.680848,8.144945
std,NaN,NaN,0 days 00:37:19.034657156,2239.034657,0.621954,3.110720,6.370754,1.950095,0.952195


In [19]:
# the mean scW is the average self-consumption in watts
selfConsumptionW = -scAn['scW'].mean()
print(selfConsumptionW)
selfConsumptionP = -scAn['scP'].mean()
print(selfConsumptionP)

-9.002763946310953
-4.395880833159645


In [66]:
sc_margin = 0

# AC Out Efficiency
Find chunks where AC in is 0 and DC in is 0. (Because of the trapazoid method, we might not need to filter AC out if its in the middle of a chunk.)

Adjust for self-consumption

In [20]:
# (allData['relay3_power'] != 0) & 
acOut_DF = allData[(allData['relay2_power'] == 0) & (allData['powerstation_inputWDC'] == 0) & (allData['powerstation_outputWDC'] == 0)] #
acOut_DF = acOut_DF[['datetime','powerstation_percentage','relay1_power','relay2_power','relay3_power']]

#acOut_DF[acOut_DF['datetime'] > (datetime.now()-timedelta(days=1))]

In [21]:
acOut_DF.describe()

,datetime,powerstation_percentage,relay1_power,relay2_power,relay3_power
count,3990,3990.000000,3990.000000,3990.0,3990.000000
mean,2025-05-07 09:12:47.707742976,69.407268,7.887469,0.0,1.105263
min,2025-05-03 00:06:15.391036,11.000000,0.000000,0.0,0.000000
25%,2025-05-04 21:45:50.872624384,64.000000,1.600000,0.0,0.000000
50%,2025-05-07 04:57:56.803738624,69.000000,1.600000,0.0,0.000000
75%,2025-05-09 20:08:32.606634752,77.000000,1.600000,0.0,0.000000
max,2025-05-12 09:29:49.423790,100.000000,202.100000,0.0,214.000000
std,NaN,10.874373,19.272304,0.0,11.155073


In [22]:
acOut_DF.tail()

,datetime,powerstation_percentage,relay1_power,relay2_power,relay3_power
6031,2025-05-12 09:20:01.444401,70.0,1.6,0.0,0.0
6032,2025-05-12 09:22:28.429441,70.0,1.6,0.0,0.0
6033,2025-05-12 09:24:56.444558,70.0,1.5,0.0,0.0
6034,2025-05-12 09:27:23.456139,70.0,1.6,0.0,0.0
6035,2025-05-12 09:29:49.423790,69.0,1.6,0.0,0.0


In [23]:
# leading or trailing rows with 0s may be an issue
acO_List = contiguousTimeChunks(acOut_DF)

acO_ListF = []        
# filter out lists with only 0 values
for l in acO_List:
    m = l['relay3_power'].mean()
    #print(m)
    pc = l['powerstation_percentage'].unique() # should have at least 2 drops in percentage
    if (m > 0 ) & (len(pc) > 2):
        acO_ListF.append(l)
    

6035


In [24]:
acO_List[-1]

,datetime,powerstation_percentage,relay1_power,relay2_power,relay3_power
6026,2025-05-12 09:07:44.413008,71.0,1.6,0.0,0.0
6027,2025-05-12 09:10:11.442880,70.0,1.6,0.0,0.0
6028,2025-05-12 09:12:37.415702,70.0,1.6,0.0,0.0
6029,2025-05-12 09:15:04.410689,70.0,5.2,0.0,0.0
6030,2025-05-12 09:17:33.449985,70.0,1.6,0.0,0.0
6031,2025-05-12 09:20:01.444401,70.0,1.6,0.0,0.0
6032,2025-05-12 09:22:28.429441,70.0,1.6,0.0,0.0
6033,2025-05-12 09:24:56.444558,70.0,1.5,0.0,0.0
6034,2025-05-12 09:27:23.456139,70.0,1.6,0.0,0.0
6035,2025-05-12 09:29:49.423790,69.0,1.6,0.0,0.0


In [25]:
acoAn = pd.DataFrame(columns=['startDT', 'endDT','len','avgW','duration','durationS','durationH','percChange','WhChange','r3_Wh','sc_Wh','eff','adjEff'])

for c in acO_ListF:
    perc_change = c.iloc[-1]['powerstation_percentage'] - c.iloc[0]['powerstation_percentage']
    print(perc_change)
    wh_change =  WhChange(perc_change, batWh) # convert percent change to Wh
    print(wh_change)
    aco_Wh = float(CONTROLS.getWh(c['relay3_power'],CONTROLS.prepWh(c)['increments'])) #energy during the chunk
    print(aco_Wh)
    dur = c.iloc[-1]['datetime'] - c.iloc[0]['datetime'] #duration of chunk
    aW = c['relay3_power'].mean() #average power during chunk
    dH = dur.total_seconds()/60/60 #chunk duration in hours
    sc = selfConsumptionW * dH #self consumption during chunk (Wh)
    print(sc)
    meteredScaler = (wh_change - sc)/aco_Wh #this is what the measured energy is multiplied by to determine battery 
    e = abs(aco_Wh / (wh_change - sc)) # efficiency accounting for self-consumption
    print(e)
    ae = min(99,e*100)*.01
    print(ae)
    df = pd.DataFrame({'startDT': [c.iloc[0]['datetime']], 'endDT': [c.iloc[-1]['datetime']],'len':[len(c)],'avgW':[aW],'duration':[dur],'durationS':[dur.total_seconds()],'durationH':[dH],'percChange':[perc_change],'WhChange':[wh_change],'r3_Wh':[aco_Wh],'sc_Wh':[sc],'eff':[e],'adjEff':[ae]})
    acoAn = pd.concat([acoAn,df], ignore_index=True)
    print('')

acoAn


-9.0
-18.432
8.809901596527778
-1.1028224784786989
0.5083854433129711
0.5083854433129711

-71.0
-145.408
123.84422110430555
-11.531052840961475
0.9250600923636642
0.9250600923636642

-22.0
-45.056000000000004
37.7128107575
-9.623012174253475
1.0643418201977564
0.99



C:\Users\alexn\AppData\Local\Temp\ipykernel_5884\4149052791.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  acoAn = pd.concat([acoAn,df], ignore_index=True)


,startDT,endDT,len,avgW,duration,durationS,durationH,percChange,WhChange,r3_Wh,sc_Wh,eff,adjEff
0,2025-05-05 14:10:39.553461,2025-05-05 14:18:00.547021,4,69.75000,0 days 00:07:20.993560,440.993560,0.122498,-9.0,-18.432,8.809902,-1.102822,0.508385,0.508385
1,2025-05-11 17:06:32.229477,2025-05-11 18:23:23.234550,32,93.93750,0 days 01:16:51.005073,4611.005073,1.280835,-71.0,-145.408,123.844221,-11.531053,0.925060,0.925060
2,2025-05-11 21:51:00.221481,2025-05-11 22:55:08.244603,27,34.62963,0 days 01:04:08.023122,3848.023122,1.068895,-22.0,-45.056,37.712811,-9.623012,1.064342,0.990000


In [26]:
acOutputEff = abs(acoAn['eff'].mean())
print(acOutputEff)

acOutputEffAdj = abs(acoAn['adjEff'].mean())
print(acOutputEffAdj)

0.8325957852914639
0.8078151785588785


In [64]:
aco_margin = 0

# AC In Efficiency
Find chunks where AC in isn't 0, DC in is 0, and AC out is 0.

In [27]:
acIn_DF = allData[(allData['relay3_power'] == 0) & (allData['relay2_power'] != 0) & (allData['powerstation_inputWDC'] == 0) & (allData['powerstation_outputWDC'] == 0) ][['datetime','powerstation_percentage','powerstation_inputWDC','relay2_power','relay3_power']]

In [28]:
acIn_DF.head()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_power,relay3_power
0,2025-05-03 00:03:32.429959,77.0,0.0,129.3,0.0
7,2025-05-03 00:20:58.409601,76.0,0.0,129.6,0.0
8,2025-05-03 00:23:25.390314,78.0,0.0,130.8,0.0
20,2025-05-03 00:53:33.433077,79.0,0.0,129.9,0.0
47,2025-05-03 02:00:52.400460,77.0,0.0,129.5,0.0


In [29]:
acIn_DF.describe()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_power,relay3_power
count,382,382.000000,382.0,370.000000,382.0
mean,2025-05-07 14:53:07.804931584,65.554974,0.0,129.760811,0.0
min,2025-05-03 00:03:32.429959,10.000000,0.0,54.500000,0.0
25%,2025-05-04 23:03:03.896534528,61.000000,0.0,129.400000,0.0
50%,2025-05-07 09:34:45.477254144,66.000000,0.0,130.150000,0.0
75%,2025-05-10 04:16:37.671925760,76.000000,0.0,130.900000,0.0
max,2025-05-12 09:05:02.433702,99.000000,0.0,133.900000,0.0
std,NaN,14.233002,0.0,4.825340,0.0


In [30]:
# leading or trailing rows with 0s may be an issue
acI_List = contiguousTimeChunks(acIn_DF)

acI_ListF = []        
# filter out lists with only 0 values
for l in acI_List:
    m = l['relay2_power'].mean()
    pc = l['powerstation_percentage'].unique() # should have at least 2 drops in percentage
    if (m > 0 ) & (len(pc) > 2):
        acI_ListF.append(l)

6025


In [31]:
aciAn = pd.DataFrame(columns=['startDT', 'endDT','len','duration','durationS','durationH','percChange','WhChange','r2_Wh','sc_Wh','eff'])

for c in acI_ListF:
    perc_change = c.iloc[-1]['powerstation_percentage'] - c.iloc[0]['powerstation_percentage'] #when charging, start percentage is substracted from end percentage
    wh_change =  WhChange(perc_change, batWh) 
    aci_Wh = float(CONTROLS.getWh(c['relay2_power'],CONTROLS.prepWh(c)['increments']))
    dur = c.iloc[-1]['datetime'] - c.iloc[0]['datetime']
    dH = dur.total_seconds()/60/60
    sc = selfConsumptionW * dH
    e = (wh_change - sc)/ aci_Wh   #when charging, self-consumption is added to change
    df = pd.DataFrame({'startDT': [c.iloc[0]['datetime']], 'endDT': [c.iloc[-1]['datetime']],
                       'len':[len(c)],'duration':[dur],'durationS':[dur.total_seconds()],
                       'durationH':[dH],'percChange':[perc_change],'WhChange':[wh_change],'r2_Wh':[aci_Wh],'sc_Wh':[sc],'eff':[e]})
    aciAn = pd.concat([aciAn,df], ignore_index=True)

aciAn

C:\Users\alexn\AppData\Local\Temp\ipykernel_5884\3544961092.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aciAn = pd.concat([aciAn,df], ignore_index=True)


,startDT,endDT,len,duration,durationS,durationH,percChange,WhChange,r2_Wh,sc_Wh,eff
0,2025-05-05 14:20:36.543824,2025-05-05 14:27:54.526534,4,0 days 00:07:17.982710,437.982710,0.121662,6.0,12.288,15.625802,-1.095293,0.856487
1,2025-05-05 14:59:59.525476,2025-05-05 15:34:17.523778,15,0 days 00:34:17.998302,2057.998302,0.571666,30.0,61.440,74.954172,-5.146576,0.888364
2,2025-05-06 22:08:53.929428,2025-05-06 22:13:54.939252,3,0 days 00:05:01.009824,301.009824,0.083614,5.0,10.240,10.901884,-0.752756,1.008335
3,2025-05-07 03:22:30.792806,2025-05-07 03:28:07.809662,3,0 days 00:05:37.016856,337.016856,0.093616,5.0,10.240,12.209696,-0.842801,0.907705
4,2025-05-07 10:03:12.152857,2025-05-07 10:08:31.821639,3,0 days 00:05:19.668782,319.668782,0.088797,5.0,10.240,11.617329,-0.799417,0.950254
5,2025-05-08 23:31:02.465138,2025-05-09 00:38:42.111161,27,0 days 01:07:39.646023,4059.646023,1.127679,58.0,118.784,147.180563,-10.152232,0.876041
6,2025-05-11 18:26:01.246108,2025-05-11 19:50:48.205968,35,0 days 01:24:46.959860,5086.959860,1.413044,73.0,149.504,181.743707,-12.721305,0.892605
7,2025-05-11 22:58:19.232772,2025-05-11 23:40:16.239432,18,0 days 00:41:57.006660,2517.006660,0.699169,41.0,83.968,87.423378,-6.294449,1.032475
8,2025-05-12 06:53:36.434160,2025-05-12 06:59:00.414632,3,0 days 00:05:23.980472,323.980472,0.089995,5.0,10.240,11.653601,-0.810200,0.948222


In [32]:
acInputEff = aciAn['eff'].mean()
acInputEff

np.float64(0.9289431247108796)

In [65]:
aci_margin = 0

# AC-in Rates

charge current at different %

In [33]:
allData.head()

,datetime,powerstation_percentage,powerstation_inputWAC,powerstation_inputWDC,powerstation_outputWAC,powerstation_outputWDC,powerstation_outputMode,powerstation_deviceType,relay1_power,relay1_current,...,relay2_voltage,relay2_status,relay2_device,relay3_power,relay3_current,relay3_voltage,relay3_status,relay3_device,mode,position
0,2025-05-03 00:03:32.429959,77.0,127.0,0.0,0.0,0.0,90.0,AC2A,1.7,0.037,...,120.8,True,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,1,NaN
1,2025-05-03 00:06:15.391036,77.0,0.0,0.0,0.0,0.0,56.0,AC2A,2.2,0.048,...,121.4,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B
2,2025-05-03 00:08:42.417400,77.0,0.0,0.0,0.0,0.0,56.0,AC2A,1.6,0.036,...,121.4,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B
3,2025-05-03 00:11:07.387912,77.0,0.0,0.0,0.0,0.0,56.0,AC2A,1.6,0.037,...,121.5,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B
4,2025-05-03 00:13:33.417699,77.0,0.0,0.0,0.0,0.0,56.0,AC2A,1.6,0.036,...,121.1,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B


In [34]:
acInRate_DF = allData[(allData['relay3_power'] == 0) & (allData['relay2_power'] != 0) & (allData['powerstation_inputWDC'] == 0) & (allData['powerstation_outputWDC'] == 0) ][['datetime','powerstation_percentage','powerstation_inputWDC','relay2_current','relay2_power','relay3_power']]

In [35]:
acInRate_DF.head()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_current,relay2_power,relay3_power
0,2025-05-03 00:03:32.429959,77.0,0.0,1.030,129.3,0.0
7,2025-05-03 00:20:58.409601,76.0,0.0,1.030,129.6,0.0
8,2025-05-03 00:23:25.390314,78.0,0.0,1.039,130.8,0.0
20,2025-05-03 00:53:33.433077,79.0,0.0,1.022,129.9,0.0
47,2025-05-03 02:00:52.400460,77.0,0.0,1.016,129.5,0.0


In [36]:
# # Scatter plot of relay2_current vs powerstation_percentage
# plt.figure(figsize=(8, 6))
# plt.scatter(df["relay2_current"], df["powerstation_percentage"], color='blue', marker='o')
# plt.xlabel("Relay2 Current (A)")
# plt.ylabel("Powerstation Percentage (%)")
# plt.title("Relay2 Current vs Powerstation Percentage")
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# PV Conversion Efficiency

DC-in to percentage to W

Find chunks where AC in is 0, DC in isn't 0, and AC out is 0.

In [37]:
dcIn_DF = allData[(allData['relay3_power'] == 0) & (allData['relay2_power'] == 0) & (allData['powerstation_inputWDC'] != 0) ][['datetime','powerstation_percentage','powerstation_inputWDC','relay2_power','relay3_power']]

In [38]:
dcIn_DF.describe()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_power,relay3_power
count,1356,1356.000000,1356.000000,1356.0,1356.0
mean,2025-05-06 22:41:42.448911360,73.696903,6.505162,0.0,0.0
min,2025-05-03 08:58:17.423362,31.000000,1.000000,0.0,0.0
25%,2025-05-03 16:52:31.386659072,66.000000,1.000000,0.0,0.0
50%,2025-05-06 16:16:35.933615872,76.000000,2.000000,0.0,0.0
75%,2025-05-09 14:53:17.611207936,79.000000,3.000000,0.0,0.0
max,2025-05-12 09:32:16.437130,91.000000,42.000000,0.0,0.0
std,NaN,8.294329,10.946059,0.0,0.0


In [39]:
# chunk and filter

# leading or trailing rows with 0s may be an issue
dcI_List = contiguousTimeChunks(dcIn_DF)

dcI_ListF = []        
# filter out lists with only 0 values
for l in dcI_List:
    m = l['powerstation_inputWDC'].mean()
    pc = l['powerstation_percentage'].unique() # should have at least 2 drops in percentage
    if (m > 1 ) & (len(pc) > 2):
        dcI_ListF.append(l)

#dcI_ListF

6036


In [55]:
dciAn = pd.DataFrame(columns=['startDT', 'endDT','len','duration','durationS','durationH','percChange','WhChange','dc_Wh','sc_Wh','eff','adjEff'])

for c in dcI_ListF:
    perc_change = c.iloc[-1]['powerstation_percentage'] - c.iloc[0]['powerstation_percentage'] #when charging, start percentage is substracted from end percentage
    # if perc_change < 0: #drop if dc in was less than sc?
    #     continue
    wh_change =  WhChange(perc_change, batWh) 
    dci_Wh = float(CONTROLS.getWh(c['powerstation_inputWDC'],CONTROLS.prepWh(c)['increments']))

    # #filter out PV inputs below 10% of PV capacity
    # if dci_Wh < (pvW * .1):
    #     continue
    dur = c.iloc[-1]['datetime'] - c.iloc[0]['datetime']
    dH = dur.total_seconds()/60/60
    sc = selfConsumptionW * dH

    if perc_change > 0:
        e = (wh_change - sc)/ dci_Wh #when charging, self-consumption is added to change
    # elif perc_change < 0:
    #     e = (wh_change - sc)/ dci_Wh #when charging, self-consumption is added to change
    else:
        continue
    ae = min(99,e*100)*.01
    df = pd.DataFrame({'startDT': [c.iloc[0]['datetime']], 'endDT': [c.iloc[-1]['datetime']],
                       'len':[len(c)],'duration':[dur],'durationS':[dur.total_seconds()],'durationH':[dH],
                       'percChange':[perc_change],'WhChange':[wh_change],'dc_Wh':[dci_Wh],'sc_Wh':[sc],'eff':[e],'adjEff':[ae]})
    dciAn = pd.concat([dciAn,df], ignore_index=True)

dciAn

C:\Users\alexn\AppData\Local\Temp\ipykernel_5884\3225512562.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dciAn = pd.concat([dciAn,df], ignore_index=True)


,startDT,endDT,len,duration,durationS,durationH,percChange,WhChange,dc_Wh,sc_Wh,eff,adjEff
0,2025-05-03 10:41:28.413573,2025-05-03 11:55:01.425946,31,0 days 01:13:33.012373,4413.012373,1.225837,7.0,14.336,28.206679,-11.035919,0.899500,0.899500
1,2025-05-03 12:28:12.395590,2025-05-03 12:45:23.383231,8,0 days 00:17:10.987641,1030.987641,0.286385,2.0,4.096,6.810204,-2.578261,0.980038,0.980038
2,2025-05-03 10:41:28.413573,2025-05-03 11:55:01.425946,31,0 days 01:13:33.012373,4413.012373,1.225837,7.0,14.336,28.206679,-11.035919,0.899500,0.899500
3,2025-05-03 12:28:12.395590,2025-05-03 12:45:23.383231,8,0 days 00:17:10.987641,1030.987641,0.286385,2.0,4.096,6.810204,-2.578261,0.980038,0.980038
4,2025-05-07 11:20:27.819170,2025-05-07 12:58:52.807202,41,0 days 01:38:24.988032,5904.988032,1.640274,14.0,28.672,48.377859,-14.767004,0.897911,0.897911
5,2025-05-10 10:31:57.163468,2025-05-10 13:04:48.164880,63,0 days 02:32:51.001412,9171.001412,2.547500,14.0,28.672,53.410122,-22.934545,0.966232,0.966232
6,2025-05-11 10:11:13.204311,2025-05-11 13:26:10.220080,80,0 days 03:14:57.015769,11697.015769,3.249171,15.0,30.720,59.192670,-29.251520,1.013158,0.990000


In [56]:
dcInputEff=dciAn['eff'].mean()
print(dcInputEff)

dcInputEffAdj=dciAn['adjEff'].mean()
print(dcInputEffAdj)

0.9480539353215727
0.9447456676240124


In [62]:
dci_margin = 0

# Inverter Model Comparison - Output Comparision

In [69]:
#convert DC to AC with PV Watts model
#https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.inverter.pvwatts.html#pvlib.inverter.pvwatts
#args: dc power input to inverter, inverter nameplate max WAC output, nameplate efficiency
invModel = []

invModel.append(pvlib.inverter.pvwatts(70, 300,0.85)/70)

In [70]:
invModel.append(pvlib.inverter.pvwatts(94, 300,0.85)/94)

In [75]:
aco_model = statistics.mean(invModel)

# Validation Report

Data to be plugged in to model

In [79]:
print(f'Self-consumption (Measured): {round(selfConsumptionW,3)} W +/- {sc_margin}W') # to do: calculate margin of error?

print(f'AC Input Efficiency (Measured): {round(acInputEff*100,3)}% +/- {aci_margin}%')

print(f'AC Output Efficiency (Measured): {round(acOutputEff*100,3)}% +/- {aco_margin}%')
print(f'AC Output Efficiency (pvlib Model): {round(aco_model*100,3)}%')

print(f'DC Input Efficiency (Measured): {round(dcInputEff*100,3)}% +/- {dci_margin}%')

Self-consumption (Measured): -9.003 W +/- 0W
AC Input Efficiency (Measured): 92.894% +/- 0%
AC Output Efficiency (Measured): 83.26% +/- 0%
AC Output Efficiency (pvlib Model): 84.613%
DC Input Efficiency (Measured): 94.805% +/- 0%


In [ ]:
# Plot charge curve